In [4]:
import os
from flask import Flask, jsonify
from supabase import create_client, Client
from time import sleep
from datetime import datetime, timedelta

app = Flask(__name__)

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

TABLE_NAME = "v2_federato_amplitude_data"

In [9]:
def get_retention_events(user_id):
    """Calculate the top 5 retention events for a given user."""
    try:
        # Get user's amplitude_id
        user_response = supabase.table("user_table").select("amplitude_id").eq("user_id", user_id).execute()
        amplitude_id = user_response.data[0]["amplitude_id"]
        
        # Get all events for this user, ordered by time
        events_response = supabase.table(TABLE_NAME)\
            .select("event_type, event_time")\
            .eq("amplitude_id", amplitude_id)\
            .order("event_time", desc=False)\
            .execute()
        
        if not events_response.data:
            return []
        
        # Convert events to a list of event types
        events = [event["event_type"] for event in events_response.data]
        
        # Calculate return frequency for each event type
        event_returns = {}
        for i, event in enumerate(events[:-1]):  # Exclude last event
            # Look at the next events to see if user returns to this event
            future_events = events[i+1:]
            if event in future_events:
                event_returns[event] = event_returns.get(event, 0) + 1
        
        # Calculate return rate (returns / total occurrences)
        event_counts = {}
        for event in events:
            event_counts[event] = event_counts.get(event, 0) + 1
            
        return_rates = {}
        for event, returns in event_returns.items():
            return_rates[event] = returns / event_counts[event]
        
        # Get top 5 events by return rate (minimum 2 occurrences)
        top_events = sorted(
            [(event, rate) for event, rate in return_rates.items() 
             if event_counts[event] >= 2],  # Filter events with at least 2 occurrences
            key=lambda x: x[1],
            reverse=True
        )[:5]
        
        # Format results
        return [{"event": event, "return_rate": rate} for event, rate in top_events]
        
    except Exception as e:
        print(f"Error calculating retention events for user {user_id}: {e}")
        return []

# Update all users in the user_table with their top retention events
def update_all_users_retention_events():
    try:
        # First, add the column directly with raw SQL
        add_column_sql = """
        DO $$ 
        BEGIN 
            IF NOT EXISTS (
                SELECT 1 
                FROM information_schema.columns 
                WHERE table_name = 'user_table' 
                AND column_name = 'top_retention_events'
            ) THEN 
                ALTER TABLE user_table ADD COLUMN top_retention_events JSONB;
            END IF;
        END $$;
        """
        
        # Execute the SQL directly through a query
        supabase.table("user_table").select("*").limit(1).execute()  # Ensure connection
        supabase.postgrest.schema("public").rpc("execute_sql", {"query": add_column_sql}).execute()
        print("Added top_retention_events column if it didn't exist")
        
        # Get all user IDs
        users_response = supabase.table("user_table").select("user_id").execute()
        
        # Process each user
        for user in users_response.data:
            user_id = user["user_id"]
            top_events = get_retention_events(user_id)
            
            # Update the user's record with their top retention events
            supabase.table("user_table")\
                .update({"top_retention_events": top_events})\
                .eq("user_id", user_id)\
                .execute()
            
            print(f"Updated retention events for user {user_id}")
            
        print("Completed updating all users' retention events")
        
    except Exception as e:
        print(f"Error updating retention events: {e}")

# Execute the update
update_all_users_retention_events()

Added top_retention_events column if it didn't exist
Updated retention events for user 1
Updated retention events for user 2
Updated retention events for user 3
Updated retention events for user 4
Updated retention events for user 5
Updated retention events for user 6
Updated retention events for user 7
Updated retention events for user 8
Updated retention events for user 9
Updated retention events for user 10
Updated retention events for user 11
Updated retention events for user 12
Updated retention events for user 13
Updated retention events for user 14
Updated retention events for user 15
Updated retention events for user 16
Updated retention events for user 17
Updated retention events for user 18
Updated retention events for user 19
Updated retention events for user 20
Updated retention events for user 21
Updated retention events for user 22
Updated retention events for user 23
Updated retention events for user 24
Updated retention events for user 25
Updated retention events for us

In [ ]:
def get_retention_events(user_id):
    """Calculate the top 5 events where users spend the most time."""
    try:
        # Get user's amplitude_id
        user_response = supabase.table("user_table").select("amplitude_id").eq("user_id", user_id).execute()
        amplitude_id = user_response.data[0]["amplitude_id"]
        
        # Get all events for this user, ordered by time
        events_response = supabase.table(TABLE_NAME)\
            .select("event_type, event_time")\
            .eq("amplitude_id", amplitude_id)\
            .order("event_time", desc=False)\
            .execute()
        
        if not events_response.data:
            return []
        
        # Calculate time spent on each event type
        event_durations = {}
        event_counts = {}
        
        for i, current_event in enumerate(events_response.data[:-1]):
            next_event = events_response.data[i + 1]
            
            # Convert string timestamps to datetime objects
            current_time = datetime.fromisoformat(current_event["event_time"].replace('Z', '+00:00'))
            next_time = datetime.fromisoformat(next_event["event_time"].replace('Z', '+00:00'))
            
            duration = (next_time - current_time).total_seconds()
            
            event_type = current_event["event_type"]
            event_durations[event_type] = event_durations.get(event_type, 0) + duration
            event_counts[event_type] = event_counts.get(event_type, 0) + 1
        
        # Calculate average duration for each event type
        avg_durations = {}
        for event_type in event_durations:
            avg_durations[event_type] = event_durations[event_type] / event_counts[event_type]
        
        # Get top 5 events by average duration (minimum 2 occurrences)
        top_events = sorted(
            [(event, duration) for event, duration in avg_durations.items() 
             if event_counts[event] >= 2],  # Filter events with at least 2 occurrences
            key=lambda x: x[1],
            reverse=True
        )[:5]
        
        # Format results (duration in seconds)
        return [{"event": event, "avg_duration": duration} for event, duration in top_events]
        
    except Exception as e:
        print(f"Error calculating retention events for user {user_id}: {e}")
        return []

# Update all users in the user_table with their top retention events
def update_all_users_retention_events():
    try:
        page_size = 1000
        last_user_id = 0
        
        while True:
            try:
                # Get next batch of users
                users_response = supabase.table("user_table")\
                    .select("user_id")\
                    .gt("user_id", last_user_id)\
                    .order("user_id")\
                    .limit(page_size)\
                    .execute()
                
                # If no more users, break
                if not users_response.data:
                    break
                
                # Process each user in this batch
                for user in users_response.data:
                    user_id = user["user_id"]
                    top_events = get_retention_events(user_id)
                    
                    # Update the user's record with their top retention events
                    supabase.table("user_table")\
                        .update({"top_retention_events": top_events})\
                        .eq("user_id", user_id)\
                        .execute()
                    
                    print(f"Updated retention events for user {user_id}")
                    last_user_id = user_id
                
                print(f"Completed batch up to user {last_user_id}")
                sleep(1)

            except Exception as batch_error:
                print(f"Batch error: {batch_error}")
                sleep(5)
                continue

        print("Completed updating all users' retention events")
        
    except Exception as e:
        print(f"Error updating retention events: {e}")

# Execute the update
update_all_users_retention_events()

Updated retention events for user 1
Updated retention events for user 2
Updated retention events for user 3
Updated retention events for user 4
Updated retention events for user 5
Updated retention events for user 6
Updated retention events for user 7
Updated retention events for user 8
Updated retention events for user 9
Updated retention events for user 10
Updated retention events for user 11
Updated retention events for user 12
Updated retention events for user 13
Updated retention events for user 14
Updated retention events for user 15
Updated retention events for user 16
Updated retention events for user 17
Updated retention events for user 18
Updated retention events for user 19
Updated retention events for user 20
Updated retention events for user 21
Updated retention events for user 22
Updated retention events for user 23
Updated retention events for user 24
Updated retention events for user 25
Updated retention events for user 26
Updated retention events for user 27
Updated re

In [ ]:
def update_user_session_metrics():
    """Add and update session metrics columns in the user_table."""
    try:
        # First, add the columns if they don't exist
        add_columns_sql = """
        DO $$ 
        BEGIN 
            -- Add average_session_time column
            IF NOT EXISTS (
                SELECT 1 FROM information_schema.columns 
                WHERE table_name = 'user_table' AND column_name = 'average_session_time'
            ) THEN 
                ALTER TABLE user_table ADD COLUMN average_session_time FLOAT;
            END IF;

            -- Add total_session_time column
            IF NOT EXISTS (
                SELECT 1 FROM information_schema.columns 
                WHERE table_name = 'user_table' AND column_name = 'total_session_time'
            ) THEN 
                ALTER TABLE user_table ADD COLUMN total_session_time FLOAT;
            END IF;

            -- Add frequency_of_sessions column
            IF NOT EXISTS (
                SELECT 1 FROM information_schema.columns 
                WHERE table_name = 'user_table' AND column_name = 'frequency_of_sessions'
            ) THEN 
                ALTER TABLE user_table ADD COLUMN frequency_of_sessions INTEGER;
            END IF;

            -- Add user_retention_30 column
            IF NOT EXISTS (
                SELECT 1 FROM information_schema.columns 
                WHERE table_name = 'user_table' AND column_name = 'user_retention_30'
            ) THEN 
                ALTER TABLE user_table ADD COLUMN user_retention_30 FLOAT;
            END IF;
        END $$;
        """
        
        # Execute the column addition
        supabase.postgrest.schema("public").rpc("execute_sql", {"query": add_columns_sql}).execute()
        print("Added new columns if they didn't exist")

        # Calculate the timestamp for 30 days ago
        thirty_days_ago = (datetime.now() - timedelta(days=30)).isoformat()

        # Process users in batches
        page_size = 1000
        last_user_id = 0
        
        while True:
            try:
                # Get next batch of users
                users_response = supabase.table("user_table")\
                    .select("user_id, amplitude_id")\
                    .gt("user_id", last_user_id)\
                    .order("user_id")\
                    .limit(page_size)\
                    .execute()
                
                if not users_response.data:
                    break
                
                # Process each user in this batch
                for user in users_response.data:
                    user_id = user["user_id"]
                    amplitude_id = user["amplitude_id"]
                    
                    # Get all events for this user in the last 30 days
                    events_response = supabase.table(TABLE_NAME)\
                        .select("event_time, session_id")\
                        .eq("amplitude_id", amplitude_id)\
                        .gte("event_time", thirty_days_ago)\
                        .order("event_time")\
                        .execute()
                    
                    if events_response.data:
                        # Group events by session
                        sessions = {}
                        for event in events_response.data:
                            session_id = event["session_id"]
                            event_time = datetime.fromisoformat(event["event_time"].replace('Z', '+00:00'))
                            
                            if session_id not in sessions:
                                sessions[session_id] = {"events": [], "duration": 0}
                            sessions[session_id]["events"].append(event_time)
                        
                        # Calculate session metrics
                        total_sessions = len(sessions)
                        total_session_time = 0
                        
                        for session in sessions.values():
                            if len(session["events"]) > 1:
                                session_duration = (max(session["events"]) - min(session["events"])).total_seconds()
                                total_session_time += session_duration
                                session["duration"] = session_duration
                        
                        # Calculate metrics
                        average_session_time = total_session_time / total_sessions if total_sessions > 0 else 0
                        frequency_of_sessions = total_sessions  # Number of sessions in past 30 days
                        
                        # Calculate retention (days active / 30 days)
                        unique_days = len(set(event_time.date() for session in sessions.values() for event_time in session["events"]))
                        user_retention_30 = unique_days / 30
                        
                        # Update user metrics
                        supabase.table("user_table")\
                            .update({
                                "average_session_time": average_session_time,
                                "total_session_time": total_session_time,
                                "frequency_of_sessions": frequency_of_sessions,
                                "user_retention_30": user_retention_30
                            })\
                            .eq("user_id", user_id)\
                            .execute()
                    
                    else:
                        # No events in last 30 days
                        supabase.table("user_table")\
                            .update({
                                "average_session_time": 0,
                                "total_session_time": 0,
                                "frequency_of_sessions": 0,
                                "user_retention_30": 0
                            })\
                            .eq("user_id", user_id)\
                            .execute()
                    
                    print(f"Updated session metrics for user {user_id}")
                    last_user_id = user_id
                
                print(f"Completed batch up to user {last_user_id}")
                sleep(1)  # Rate limiting

            except Exception as batch_error:
                print(f"Batch error: {batch_error}")
                sleep(5)  # Longer sleep on error
                continue

        print("Completed updating all users' session metrics")
        
    except Exception as e:
        print(f"Error updating session metrics: {e}")

# Execute the update
update_user_session_metrics()

Added new columns if they didn't exist
Updated session metrics for user 1
Updated session metrics for user 2
Updated session metrics for user 3
Updated session metrics for user 4
Updated session metrics for user 5
Updated session metrics for user 6
Updated session metrics for user 7
Updated session metrics for user 8
Updated session metrics for user 9
Updated session metrics for user 10
Updated session metrics for user 11
Updated session metrics for user 12
Updated session metrics for user 13
Updated session metrics for user 14
Updated session metrics for user 15
Updated session metrics for user 16
Batch error: {'code': '57014', 'details': None, 'hint': None, 'message': 'canceling statement due to statement timeout'}
Updated session metrics for user 17
Updated session metrics for user 18
Updated session metrics for user 19
Updated session metrics for user 20
Updated session metrics for user 21
Updated session metrics for user 22
Updated session metrics for user 23
Updated session metri

In [ ]:
def update_user_active_periods():
    """Add and update top 5 daily active time periods for each user."""
    try:
        # First, add the column if it doesn't exist
        add_column_sql = """
        DO $$ 
        BEGIN 
            IF NOT EXISTS (
                SELECT 1 FROM information_schema.columns 
                WHERE table_name = 'user_table' AND column_name = 'daily_active_periods'
            ) THEN 
                ALTER TABLE user_table ADD COLUMN daily_active_periods JSONB;
            END IF;
        END $$;
        """
        
        # Execute the column addition
        supabase.postgrest.schema("public").rpc("execute_sql", {"query": add_column_sql}).execute()
        print("Added daily_active_periods column if it didn't exist")

        # Process users in batches
        page_size = 1000
        last_user_id = 0
        
        while True:
            try:
                # Get next batch of users
                users_response = supabase.table("user_table")\
                    .select("user_id, amplitude_id")\
                    .gt("user_id", last_user_id)\
                    .order("user_id")\
                    .limit(page_size)\
                    .execute()
                
                if not users_response.data:
                    break
                
                # Process each user in this batch
                for user in users_response.data:
                    user_id = user["user_id"]
                    amplitude_id = user["amplitude_id"]
                    
                    # Get all events for this user
                    events_response = supabase.table(TABLE_NAME)\
                        .select("event_time")\
                        .eq("amplitude_id", amplitude_id)\
                        .execute()
                    
                    if events_response.data:
                        # Initialize hour counters
                        hour_counts = {str(i).zfill(2): 0 for i in range(24)}
                        
                        # Count events by hour
                        for event in events_response.data:
                            event_time = datetime.fromisoformat(event["event_time"].replace('Z', '+00:00'))
                            hour = str(event_time.hour).zfill(2)
                            hour_counts[hour] += 1
                        
                        # Get only hours with events
                        active_hours = [(hour, count) for hour, count in hour_counts.items() if count > 0]
                        
                        # Sort by count and get top 5
                        top_5_periods = sorted(active_hours, key=lambda x: x[1], reverse=True)[:5]
                        
                        # Format the results
                        active_periods = [
                            {
                                "hour": hour,
                                "count": count,
                                "period": f"{hour}:00-{str((int(hour) + 1) % 24).zfill(2)}:00"
                            }
                            for hour, count in top_5_periods
                        ]
                        
                        # Update user metrics
                        supabase.table("user_table")\
                            .update({"daily_active_periods": active_periods})\
                            .eq("user_id", user_id)\
                            .execute()
                    
                    else:
                        # No events
                        supabase.table("user_table")\
                            .update({"daily_active_periods": []})\
                            .eq("user_id", user_id)\
                            .execute()
                    
                    print(f"Updated active periods for user {user_id}")
                    last_user_id = user_id
                
                print(f"Completed batch up to user {last_user_id}")
                sleep(1)  # Rate limiting

            except Exception as batch_error:
                print(f"Batch error: {batch_error}")
                sleep(5)  # Longer sleep on error
                continue

        print("Completed updating all users' active periods")
        
    except Exception as e:
        print(f"Error updating active periods: {e}")

# Execute the update
update_user_active_periods()
